In [1]:
import numpy as np
import random
import math
from tqdm import tqdm
import os
import pickle
import ctypes

In [ ]:
# Prevents the screen saver, sleep, and monitor power off
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)

def allow_sleep():
    ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)

In [2]:
# My activation function of choice. Sigmoid returns a value between 1 and 0.
def sigmoid(x) -> float:
    return 1 / (1 + np.exp(-x))

# Applies the sigmoid function to every value in the vector.
def vector_sigmoid(vector) -> np.ndarray:
    sig_vector = np.vectorize(sigmoid)(vector)
    return sig_vector

# Derivative of the sigmoid function.
def sigmoid_derivative(x) -> float:
    return sigmoid(x)*(1-sigmoid(x))

In [3]:
# Calculates the values of a single layer.
def calculate_layer(layer : int) -> None:
    global weights, biases, activations, weighted_inputs
    # z(L) = b(L) + W(L) * a(L-1)
    #print("biases[layer]: ", biases[layer])
    weighted_input = biases[layer] + (weights[layer] @ activations[layer-1])
    
    weighted_inputs[layer] = weighted_input
    activated = vector_sigmoid(weighted_input)
    activations[layer] = activated


# Loops through every layer and calculates their activation values.
def calculate_output() -> None:
    global layers, activations
    for i in range(1,len(layers)):
        calculate_layer(i)


In [4]:
# Calculates the derivative of the chain rule.
def calculate_chain_derivative(is_parent, layer, row, column) -> float:
    global weights, weighted_inputs, activations, expected_output
    sum = 0
    if layer == len(activations) - 1:
        cost_derivative = 2 * (activations[layer][row] - expected_output[row])
        
        sum = cost_derivative
    else:
        for i in range(len(activations[layer+1])):
            sum += calculate_chain_derivative(False, layer + 1, i, row)
    weighted_input = weighted_inputs[layer][row]
    activation_derivative = sigmoid_derivative(weighted_input)
    if is_parent:
        return sum * activation_derivative
    else:
        weighted_input_derivative = weights[layer][row][column]
        return sum * activation_derivative * weighted_input_derivative

In [5]:
# Calculates the derivative of the bias.
def calculate_bias_derivative(is_parent, layer, row, column) -> float: # Literally just a useless function.
    chain_derivative = calculate_chain_derivative(is_parent, layer, row, column)
    return chain_derivative

# Calculates the derivative of the weight.
def calculate_weight_derivative(is_parent, layer, row, column) -> float:  
    chain_derivative = calculate_chain_derivative(is_parent, layer, row, column)
    result = chain_derivative * activations[layer-1][column]
    return result

In [6]:

def backpropagate() -> None:
    global weights, biases, activations, weighted_inputs, expected_output, weight_gradient, bias_gradient

    for layer in range(1, len(weights)):
        for row in range(len(weights[layer])):
            for column in range(len(weights[layer][row])):
                weight_gradient[layer][row][column] = calculate_weight_derivative(True, layer, row, column)
    
    for layer in range(1, len(biases)):
        for row in range(len(biases[layer])):
            bias_gradient[layer][row] = calculate_bias_derivative(True, layer, row, column)

In [7]:
def set_input(input : np.ndarray) -> None:
    global activations
    activations[0] = input

def set_expected_output(output : np.ndarray) -> None:
    global expected_output
    expected_output = output

In [8]:
def next_sample():
    global index, data
    #print("Index: ", index)
    sample = data[index]
    set_input(sample[0])
    set_expected_output(sample[1])
    index += 1

In [9]:
def learn() -> None:
    next_sample()
    calculate_output()
    backpropagate()

In [ ]:
def save_network() -> None:
    global weights, biases
    with open("weights.pkl", "wb") as file:
        pickle.dump(weights, file)
    with open("biases.pkl", "wb") as file:
        pickle.dump(biases, file)

In [10]:
# TODO: make it loop around the test data if it reaches the end.
def mini_batch_gradient_descent(epochs: int, batch_size: int, lr: float, save_frequency: int) -> None:
    prevent_sleep()
    global weight_gradient, bias_gradient, index, weights, biases
    index = 0
    for epoch in tqdm(range(epochs), desc="Epoch", leave=None):
        batch_weights = [np.zeros_like(layer) for layer in weight_gradient]
        batch_weights = np.array(batch_weights, dtype=object)
        batch_biases = [np.zeros_like(layer) for layer in bias_gradient]
        batch_biases = np.array(batch_biases, dtype=object)

        missed = 0
        for i in range(batch_size):
            if i + index >= len(data):
                missed += 1
                break
            learn()
            #print("batch_weights: ", batch_weights)
            #print("weight_gradient: ", weight_gradient)
            batch_weights += weight_gradient
            batch_biases += bias_gradient
        
        batch_weights = np.array([x / (batch_size - missed) for x in batch_weights],dtype=object)
        #weights -= batch_weights
        for w, b in zip(weights, batch_weights):
            w -= b * lr
        
        
        batch_biases = np.array([x / (batch_size - missed) for x in batch_biases],dtype=object)
        #biases -= batch_biases
        for b, b2 in zip(biases, batch_biases):
            b -= b2 * lr

        if epoch % save_frequency == 0:
            save_network()
    allow_sleep()


In [11]:
# Check if the formatted data file exists
if os.path.exists("formatted_data.pkl"):
    print("Formatted data file already exists. Loading data...")
    with tqdm(total=1, desc="Loading data") as pbar:
        with open("formatted_data.pkl", "rb") as file:
            data = pickle.load(file)
        pbar.update(1)
else:
    print("Formatted data file does not exist. Formatting data...")
    # Your existing code for formatting the data
    
    with open("mnist_train.csv", "r") as file:
        lines = file.readlines()
        data = []
        total_lines = len(lines)
        for line in tqdm(lines, total=total_lines, desc="Processing data"):
            values = line.strip().split(",")
            label = int(values[0])
            image = np.array([float(x)/255 for x in values[1:]], dtype=np.float32)
            if len(image) != 784: print("Error! Image wrong size: " + len(image))
            expected_output = np.zeros(10)
            expected_output[label] = 1
            example = [image, expected_output]
            data.append(example)

        # Save the training data into a file
        with open("formatted_data.pkl", "wb") as file:
            pickle.dump(data, file)
        print("Training data saved into formatted data file.")


#print(data[0])


Formatted data file already exists. Loading data...


Loading data: 100%|██████████| 1/1 [00:00<00:00,  2.67it/s]


In [12]:
# Turn off dumb warnings
#np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
layers = [784, 16, 16, 10]

weights = [0] + [np.random.uniform(-1,1,size=(layers[i], layers[i-1])) for i in range(1, len(layers))]
weigths = np.array(weights,dtype=object)

biases = [0] + [np.random.uniform(-1, 1, size=layers[i]) for i in range(1, len(layers))]
biases = np.array(biases, dtype=object)

weighted_inputs = [0] + [np.zeros((layers[i], 1)) for i in range(1, len(layers))]
weighted_inputs = np.array(weighted_inputs, dtype=object)

activations = [np.zeros((layers[i], 1)) for i in range(0, len(layers))]
weight_gradient = [np.zeros_like(layer) for layer in weights]
weight_gradient = np.array(weight_gradient, dtype=object)
bias_gradient = [np.zeros_like(layer) for layer in biases]
bias_gradient = np.array(bias_gradient, dtype=object)

mini_batch_gradient_descent(200, 10, 0.005)
# If you get a divide by zero error, it's probably because the batch size and epochs is too large.


Epoch: 100%|██████████| 200/200 [9:31:43<00:00, 171.52s/it]     

Weights and biases saved into files.


Now that the NN has been trained, we can test its accuracy.

In [14]:
def test() -> None:
    global activations, test_data
    correct = 0
    for sample in test_data:
        set_input(sample[0])
        set_expected_output(sample[1])
        calculate_output()
        output = activations[-1]
        if np.argmax(output) == np.argmax(sample[1]):
            correct += 1
    print("Correct: ", correct, "/", len(test_data))
    print("Accuracy: ", correct/len(test_data)*100, "%")

In [13]:
# Check if the formatted data file exists
if os.path.exists("formatted_test_data.pkl"):
    print("Formatted test data file already exists. Loading data...")
    with tqdm(total=1, desc="Loading data") as pbar:
        with open("formatted_test_data.pkl", "rb") as file:
            test_data = pickle.load(file)
        pbar.update(1)
else:
    print("Formatted test data file does not exist. Formatting data...")
    # Your existing code for formatting the data
    
    with open("mnist_test.csv", "r") as file:
        lines = file.readlines()
        test_data = []
        total_lines = len(lines)
        for line in tqdm(lines, total=total_lines, desc="Processing data"):
            values = line.strip().split(",")
            label = int(values[0])
            image = np.array([float(x)/255 for x in values[1:]], dtype=np.float32)
            if len(image) != 784: print("Error! Image wrong size: " + len(image))
            expected_output = np.zeros(10)
            expected_output[label] = 1
            example = [image, expected_output]
            test_data.append(example)

        # Save the training data into a file
        with open("formatted_test_data.pkl", "wb") as file:
            pickle.dump(test_data, file)
        print("Training data saved into formatted test data file.")

Formatted test data file does not exist. Formatting data...


Processing data: 100%|██████████| 10000/10000 [00:01<00:00, 5092.30it/s]


Training data saved into formatted test data file.


In [15]:
test()

Correct:  1046 / 10000
Accuracy:  10.459999999999999 %


In [16]:
print(weights)

[0, array([[ 0.06937077, -0.84508992,  0.38098064, ..., -0.29113902,
        -0.16481954, -0.61047532],
       [-0.53709845,  0.02162415,  0.32540907, ...,  0.05456196,
         0.45044997, -0.73815714],
       [ 0.29402003, -0.84784307,  0.18087987, ...,  0.53424392,
        -0.76786185,  0.28611547],
       ...,
       [ 0.44055076,  0.84104061,  0.50117378, ...,  0.2252591 ,
        -0.18963522,  0.07030325],
       [-0.73396905,  0.82878698,  0.28206588, ...,  0.40450718,
        -0.07328773,  0.00426636],
       [ 0.61512547, -0.60952177,  0.01999299, ..., -0.54131904,
        -0.35026074,  0.03996377]]), array([[ 0.41216561, -0.26690612,  0.71384004,  0.07303268,  0.5756867 ,
         0.8181677 ,  0.09272674,  0.83188007,  0.2563646 , -0.05931308,
         0.5741621 , -0.11587914, -0.93894792,  0.05942882,  0.26541549,
         0.97803597],
       [-0.62014687,  0.97267346,  0.56770552, -0.61508529,  0.69505902,
        -0.33561932, -0.28380962,  0.22250728, -0.83866526,  0.36143